## ACLED Historic Data Updates
ACLED Data can be downloaded via their website here, https://www.acleddata.com/data/. This python notebook will walk you through automating this process. It will download and extract the data in XLSX format, process it, and then use it to update a hosted feature layer in ArcGIS Online.

In [ ]:
import requests
import zipfile
import io
import json
import getpass
import pandas as pd
from IPython.display import clear_output

from arcgis.gis import GIS

### Setup your connection to ArcGIS Online

In [ ]:
username = input('Username: ')
password = getpass.getpass('Password: ')
arcgis_url = 'https://www.arcgis.com'

# create your `gis` instance 
gis = GIS(arcgis_url, username, password)

print ('succesfully logged in.')

### Download and extract the latest historical zip file from ACLED

Set the `region` key to one of:
- Africa
- Middle East
- South and South East Asia


In [ ]:
download_url = 'https://www.acleddata.com/download/{}/'

region = 'Middle East'

if region == 'Middle East':
    url_region_id = '2915'
elif region == 'Africa':
    url_region_id = '2909'
elif region == 'South and South East Asia':
    url_region_id = '2912'

download_url = download_url.format(url_region_id)

print ('downloading historic zip file for {} ..'.format(region))
r = requests.get(download_url)
z = zipfile.ZipFile(io.BytesIO(r.content))

print ('extracting zip file ..')
archive_filename = z.namelist()[0]

z.extractall()
print ('done')

### Use Pandas to read in the excel file

In [ ]:
print ('reading xlsx into pandas ..')
excel_df = pd.read_excel(io=archive_filename, sheet_name=0)

### Add in two more columns
These columns help us identify the source for each record that will make it easier to make future updates.

In [ ]:
print ('adding new region column to pandas dataframe ..')
excel_df.insert(len(excel_df.columns), 'REGION_FROM_FILE', region)
excel_df.insert(len(excel_df.columns), 'ISO3', None)

Set the ISO3 code column values based on the first 3 characters of the EVENT_ID_CNTY column. _courtesy of [Stack Overflow](https://stackoverflow.com/questions/12604909/pandas-how-to-change-all-the-values-of-a-column/12605055#12605055)_

In [ ]:
excel_df['ISO3'] = excel_df['EVENT_ID_CNTY'].apply(lambda x: x[:3])

### Get the Historic layer ArcGIS Online Item
In our situation, the historic layer is the second (index of 1) layer in the Feature service

In [ ]:
historic_item_id = 'a6e330c275dd421a9a3dda6e8e546f3d'
historic_item = gis.content.get(itemid=historic_item_id)
fl = historic_item.layers[1]

### Delete existing region features
When using a hosted feature service in ArcGIS Online with a large amount of features, it's best to chunk up any editing jobs into reasonable sized requests. Here we will delete 1,000 features at a time, deleting existing features by object id.

In [ ]:
res = fl.query(where='REGION_FROM_FILE = \'{}\''.format(region), return_ids_only=True)

This handy python function (from [Stack Overflow](https://stackoverflow.com/a/312464)) to split up the data into chunks of 1,000.

In [ ]:
n = 1000
l = res['objectIds']
delete_chunks = [l[i:i+n] for i in range(0, len(l), n)]

In [ ]:
total_chunks = len(delete_chunks)
for i, chunk in enumerate(delete_chunks):
    oids = ','.join(map(str, chunk))
    try:
        delete_response = fl.delete_features(deletes=oids)
        print ('deleted chunk {} of {}'.format(i+1,total_chunks))
        clear_output(wait=True)
    except Exception as e:
        print ('error deleting features')

### Enable this line for testing with a subset of the data

In [ ]:
# excel_df = excel_df.head(8000)

### Split the dataset into chunks of 1,000 for adding to the feature layer

In [ ]:
n = 1000
list_df = [excel_df[i:i+n] for i in range(0,excel_df.shape[0],n)]

### Construct the features
Pandas dataframe has a handy `to_json()` method we can use here. From there, we just construct a feature object and stash it into an array

In [ ]:
feature_chunks = []
for df in list_df:
    df_json_string = df.to_json(orient='records')
    df_json = json.loads(df_json_string)
    features = []
    for rec in df_json:
        feature = {
            'attributes': rec,
            'geometry': {
                'x': rec['LONGITUDE'],
                'y': rec['LATITUDE']
            }
        }
        features.append(feature)
    
    feature_chunks.append(features)

### Make the edits to the feature layer
Here we loop through each chunk of 1,000 features and apply those edits

In [ ]:
chunk_length = len(feature_chunks)
for i, chunk in enumerate(feature_chunks):
    try:
        fl.edit_features(adds=chunk)
        print ('added chunk {} of {}'.format(i+1, chunk_length))
        clear_output(wait=True)
    except Exception as e:    
        print ('error adding chunk')


### Optionally send an email when it's all done

In [ ]:
import smtplib
from email.message import EmailMessage

mail_server = ''
email_address = ''

msg = EmailMessage()
msg.set_content('hi there. your python script is done adding the features for the {} region'.format(region))
msg['Subject'] = 'Done updating ACLED historic layer'
msg['from'] = email_address
msg['to'] = email_address

s = smtplib.SMTP(mail_server)
s.send_message(msg)
s.quit()